In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                             'username': '186299u15',
                             'password': 'APMTeDoW14'}}

b = 0
while True:
    b += 1

    try:
        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(),
                             #proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()

            if json_['status'] == 200:
                key = json_['apiKey']
                break
    except KeyboardInterrupt:
        break
    except:
        continue

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./file/1.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =


work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_error, key

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            output, i = pd.DataFrame(), 1
            while i <= input_.loc[a, 'Page']:
                url = input_.loc[a, 'Url']
            
                dict_param = {}
                list_param = url.split('#@')[0].split('?')[1].split('&') if '?' in url else []
                for param in list_param:
                    dict_param[param.split('=')[0]] = param.split('=')[1]
                
                data = {'getArticles': {'articleCountry': 'DE',
                                        'articleStatusIds': [],
                                        'criteriaFilters': [],
                                        'dataSupplierIds': [int(dict_param['brands'])],
                                        'filterQueries':  ['(dataSupplierId NOT IN (4978,4982))'],
                                        'genericArticleIds': [int(dict_param['groups'])],
                                        'includeAccessoryArticles': True,
                                        'includeAll': True,
                                        'includeArticleCriteria': True,
                                        'includeArticleLogisticsCriteria': True,
                                        'includeArticleStatusFacets': True,
                                        'includeArticleText': True,
                                        'includeComparableNumbers': True,
                                        'includeCriteriaFacets': True,
                                        'includeDataSupplierFacets': True,
                                        'includeGTINs': True,
                                        'includeGenericArticleFacets': True,
                                        'includeGenericArticles': True,
                                        'includeImages': True,
                                        'includeLinkages': True,
                                        'includeLinks': True,
                                        'includeMisc': True,
                                        'includeOEMNumbers': True,
                                        'includePDFs': True,
                                        'includePartsListArticles': True,
                                        'includePrices': True,
                                        'includeReplacedByArticles': True,
                                        'includeReplacesArticles': True,
                                        'includeTradeNumbers': True,
                                        'lang': 'en',
                                        'linkagesPerPage': 100,
                                        'page': i,
                                        'perPage': 1_000,
                                        'provider': 40_000,
                                        'sort': [{'field': 'articleNumber', 'direction': 'asc'}]}}
                
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                             data=json.dumps(data),
                                             headers=get_header(**{'x-api-key': key}),
                                             #proxies=get_proxy(),
                                             timeout=(10, 10))
                            
                        if resp.status_code == 200:
                            json_ = resp.json()
                
                            if json_['status'] == 401 and json_['statusText'].strip() == 'Access not allowed':
                                while True:
                                    try:
                                        data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                                                                     'username': '186299u15',
                                                                     'password': 'APMTeDoW14'}}
                                        
                                        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                                                             data=json.dumps(data),
                                                             headers=get_header(),
                                                             #proxies=get_proxy(),
                                                             timeout=(10, 10))
                                
                                        if resp.status_code == 200:
                                            json_ = resp.json()
                                
                                            if json_['status'] == 200:
                                                key = json_['apiKey']
                                                break
                                    except KeyboardInterrupt:
                                        break
                                    except:
                                        continue
                            elif json_['status'] == 200:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue
            
                # = = = = = = = = = = = = = = =
            
                list_part = json_['articles']
            
                # = = = = = = = = = = = = = = =
            
                if not list_part:
                    break
            
                # = = = = = = = = = = = = = = =
            
                list_part_number = [part['articleNumber'].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
            
                list_brand = [part['mfrName'].strip() for part in list_part]
            
                # = = = = = = = = = = = = = = =
            
                list_additional_description = [part['misc']['additionalDescription'].strip() if 'additionalDescription' in part['misc'] else ''for part in list_part]
            
                # = = = = = = = = = = = = = = =
            
                list_status = [part['misc']['articleStatusDescription'].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
            
                list_packing_unit = [part['misc']['quantityPerPackage'] if 'quantityPerPackage' in part['misc'] else '' for part in list_part]
            
                # = = = = = = = = = = = = = = =
            
                list_quantity_per_packing_unit = [part['misc']['quantityPerPartPerPackage'] if 'quantityPerPartPerPackage' in part['misc'] else '' for part in list_part]
            
                # = = = = = = = = = = = = = = =
            
                list_mandatory_material_certification = [part['misc']['hasMandatoryMaterialCertification'] if 'hasMandatoryMaterialCertification' in part['misc'] else '' for part in list_part]
            
                # = = = = = = = = = = = = = = =
            
                list_remanufactured = [part['misc']['isRemanufacturedPart'] if 'isRemanufacturedPart' in part['misc'] else '' for part in list_part]
            
                # = = = = = = = = = = = = = = =

                list_group = [part['genericArticles'][0]['genericArticleDescription'] for part in list_part]

                # = = = = = = = = = = = = = = =
            
                list_part_code = [part['genericArticles'][0]['legacyArticleId'] for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_dict_information, list_oem, list_dict_criteria, list_dict_src, list_dict_price = [], [], [], [], []
                for part in list_part:
                    dict_information, list_information = {}, [' '.join([text.strip() for text in list_text['text']]) for list_text in part['articleText']]
                    for j, information in zip(range(len(list_information)), list_information):
                        dict_information[str(j)] = {part['articleText'][j]['informationTypeDescription'].strip(): information}
                    
                    list_dict_information.append(dict_information)
                
                    # = = = = = = = = = = = = = = =
                    
                    list_oem_temp = [oem['articleNumber'].strip() for oem in part['oemNumbers']]
                    list_oem_temp = [oem.replace(' ', '') for oem in list_oem_temp]
                    
                    list_oem.append(';'.join(sorted(list(set(list_oem_temp)), key=list_oem_temp.index)))
            
                    # = = = = = = = = = = = = = = =
            
                    dict_criteria, list_criteria = {}, part['articleCriteria']
                    for j, criteria in zip(range(len(list_criteria)), list_criteria):
                        dict_criteria[str(j)] = {criteria['criteriaAbbrDescription'].strip() if 'criteriaAbbrDescription' in criteria else criteria['criteriaDescription'].strip(): f'''{criteria['formattedValue'].strip()} {criteria['criteriaUnitDescription'].strip()}''' if 'criteriaUnitDescription' in criteria else criteria['formattedValue'].strip()}
                
                    list_dict_criteria.append(dict_criteria)
            
                    # = = = = = = = = = = = = = = =
            
                    dict_src, list_src = {}, part['images']
                    for j, src in zip(range(len(list_src)), list_src):
                        dict_src[str(j)] = src['imageURL3200'].strip()
                
                    list_dict_src.append(dict_src)
            
                    # = = = = = = = = = = = = = = =
            
                    dict_price, list_price = {}, part['prices']
                    for j, price in zip(range(len(list_price)), list_price):
                        dict_price[str(j)] = {}
                        dict_price[str(j)]['price_type'] = price['kindOfPriceDescription'].strip()
                        dict_price[str(j)]['price_unit'] = price['priceUnitDescription'].strip()
                        dict_price[str(j)]['quantity_unit'] = price['quantityUnitDescription'].strip()
                        dict_price[str(j)]['price'] = ((price['currencyCode'].strip() if 'currencyCode' in price else '') + ' ' + (str(price['price']) if 'price' in price else '')).strip()
                        dict_price[str(j)]['discount'] = price['isDiscount']
                        
                    list_dict_price.append(dict_price)
            
                # = = = = = = = = = = = = = = =
            
                list_gtin = [';'.join([gtin.strip() for gtin in part['gtins']]) for part in list_part]
            
                # = = = = = = = = = = = = = = =
            
                df_temp = pd.DataFrame({'No': [j + 1 + i * 1_000 - 1_000 for j in range(len(list_part))],
                                        'Brand': list_brand,
                                        'Group': list_group,
                                        'Part Number': list_part_number,
                                        'Remanufactured': list_remanufactured,
                                        'Status': list_status,
                                        'Additional Description': list_additional_description,
                                        'GTIN/EAN': list_gtin,
                                        'OEM': list_oem,
                                        'Mandatory Material Certification': list_mandatory_material_certification,
                                        'Packing Unit': list_packing_unit,
                                        'Quantity Per Packing Unit': list_quantity_per_packing_unit,
                                        'Json_Information': [json.dumps(dict_information) for dict_information in list_dict_information],
                                        'Json_Criteria': [json.dumps(dict_criteria) for dict_criteria in list_dict_criteria],
                                        'Json_Price': [json.dumps(dict_price) for dict_price in list_dict_price],
                                        'Picture': '',
                                        'Url': [f'''https://web.tecalliance.net/tecdocsw/en/parts/{int(dict_param['brands'])}/{part_number}/detail?groups={int(dict_param['groups'])}''' for part_number in list_part_number],
                                        'Json_Src': [json.dumps(dict_src) for dict_src in list_dict_src],
                                        'Part Code': list_part_code})

                output = pd.concat([output, df_temp], ignore_index=True).fillna('')
            
                # = = = = = = = = = = = = = = =

                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
                
                print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']} > Page {i}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                # = = = = = = = = = = = = = = =
            
                i += 1
                        
            # = = = = = = = = = = = = = = =

            output = output.sort_values(by=['No'],
                                        ascending=[True],
                                        ignore_index=True)

            output.to_excel(f'''./file/2.part/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/part_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[尝试次数：1] - 2.https://web.tecalliance.net/tecdocsw/en/parts/search?brands=244&groups=1361 > Page 1
[剩余数量：0] - [当前时间：16:39:03]

[尝试次数：1] - 1.https://web.tecalliance.net/tecdocsw/en/parts/search?brands=101&groups=247 > Page 1
[剩余数量：0] - [当前时间：16:39:04]

[尝试次数：1] - 1.https://web.tecalliance.net/tecdocsw/en/parts/search?brands=101&groups=247 > Page 2
[剩余数量：0] - [当前时间：16:39:06]

输出ing...



Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4355.46it/s]

Done ~
